# Setup

In [1]:
import gc
import random

import astropy.units as u
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sunpy.map
import sunpy.visualization.colormaps.color_tables as ct
from astropy.visualization import AsinhStretch, ImageNormalize
from IPython.display import clear_output, display
from matplotlib.patches import Rectangle
from sunpy.coordinates import frames
from tqdm import tqdm
import json
import random
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output

from contextlib import contextmanager

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from Library.Processing import *
from Library.IO import *
from Library.Model import *
from Library.Metrics import *
from Library.Config import *
from Library.CH import *
from Library.Plot import *
from Models import load_date_range


/opt/homebrew/Caskroom/miniconda/base/envs/icme3.12-metal/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:
pd.set_option("display.width", 10000)
pd.set_option("display.max_colwidth", None)

# Data Processing

In [5]:
df = pd.read_parquet(paths["artifact_root"] + "Paths.parquet")

In [6]:
ARCH_ID = "A0"
DATE_ID = "D0"

date_range = load_date_range(ARCH_ID, DATE_ID)
train_df, val_df = date_range.select_pairs(df)


In [7]:
# set-wise subtraction
if val_df is None:
    inf_df = df.loc[~df.index.isin(train_df.index)]
else:
    combined_df = pd.concat([train_df, val_df])
    inf_df = df.loc[~df.index.isin(combined_df.index)]


# Model

## Training

In [8]:
# train_model(train_df)

In [9]:
model = load_trained_model(ARCH_ID, DATE_ID)


/Users/aosh/Developer/helio-n/Outputs/Models/A0D0.keras


I0000 00:00:1768129315.919308 3050831 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1768129315.919333 3050831 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Plotting

In [10]:
def plot_sdo(
    row,
    postprocessing="P0",
    oval=None,
    show_fits=True,
    pmap=None,
):
    """
    Double plotter wrapper around plot_ch_map().
    Keeps the original signature.

    - Left: U-Net (helio-n)
    - Right: IDL
    """
    print(row)
    # Keep existing diagnostic print; assumes these globals exist in your notebook/module
    params = (
        postprocessing
        if isinstance(postprocessing, dict)
        else get_postprocessing_params(postprocessing)
    )
    print_distance(row, model, params)

    # Keep old behavior: compute oval once and reuse
    if oval is None:
        oval = generate_omask(row)

    m = sunpy.map.Map(row.fits_path)

    # Map old "smoothing_params" to the string key plot_ch_map expects.
    # This preserves call compatibility without changing plot_ch_map's signature.
    # If you have a canonical mapping util, swap this block out.

    fig = plt.figure(figsize=((TARGET_PX / DPI) * 2.1, TARGET_PX / DPI))
    ax1 = fig.add_subplot(
        1, 2, 1, projection=m
    )  # let plot_ch_map create/use WCS projection internally
    ax2 = fig.add_subplot(1, 2, 2, projection=m)

    # Left panel: U-Net
    plot_ch_map(
        row,
        source="unet",
        model=model,  # required if no pmap_path/pmap on row
        pmap=pmap,  # plot_ch_map will load/compute if needed
        postprocessing=postprocessing,
        oval=oval,
        show_fits=show_fits,
        multiplot_ax=ax1,
        set_title=False,
    )
    ax1.set_title("helio-n (U-Net)")

    # Right panel: IDL
    plot_ch_map(
        row,
        source="idl",
        model=None,
        pmap=None,
        postprocessing=postprocessing,  # ignored for IDL, harmless to pass
        oval=oval,
        show_fits=show_fits,
        multiplot_ax=ax2,
        set_title=False,
    )
    ax2.set_title("IDL")

    plt.tight_layout()
    plt.show()

In [11]:
_SUPPRESS_REDRAW = 0


@contextmanager
def suppress_redraw():
    global _SUPPRESS_REDRAW
    _SUPPRESS_REDRAW += 1
    try:
        yield
    finally:
        _SUPPRESS_REDRAW -= 1


# -------------------------
# 0) Postprocessing presets
# -------------------------
PP_DIR = Path("./Config/Postprocessing").resolve()


def list_json_presets(pp_dir: Path):
    if not pp_dir.exists():
        return ["Custom"], {"Custom": None}

    files = sorted(
        [p for p in pp_dir.iterdir() if p.is_file() and p.suffix.lower() == ".json"]
    )
    names = [p.stem for p in files]
    mapping = {p.stem: p for p in files}

    # Ensure Custom exists as an option even if file missing
    if "Custom" not in mapping:
        names = ["Custom"] + names
        mapping["Custom"] = pp_dir / "Custom.json"

    # Make Custom default and first in list
    if "Custom" in names:
        names = ["Custom"] + [n for n in names if n != "Custom"]

    return names, mapping


PRESET_NAMES, PRESET_PATHS = list_json_presets(PP_DIR)
DEFAULT_PRESET = "P1" if "P1" in PRESET_NAMES else PRESET_NAMES[0]


def load_params_from_file(path: Path, fallback: dict):
    try:
        if path is None:
            return dict(fallback)
        with path.open("r", encoding="utf-8") as f:
            data = json.load(f)
        # Only take known keys; fallback for missing
        out = dict(fallback)
        for k in ["threshold", "closing_radius", "min_size", "hole_size"]:
            if k in data:
                out[k] = data[k]
        return out
    except Exception:
        return dict(fallback)


def write_params_to_custom(path: Path, params: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(params, f, indent=2, sort_keys=True)


# --------------------------------
# 1) Register your dataframes here
# --------------------------------
dfs = {
    "train": train_df,
    "inference": inf_df,
}

# ----------------
# 2) Widgets
# ----------------
full_row = widgets.Layout(width="100%")
row_wrap = widgets.Layout(
    display="flex", flex_flow="row wrap", align_items="center", width="100%"
)
checkbox_layout = widgets.Layout(margin="0 0 0 12px")


df_selector = widgets.RadioButtons(
    options=list(dfs.keys()),
    value="inference",
    description="DataFrame:",
    layout=widgets.Layout(margin="0 8px 0 0"),
)

idx_slider = widgets.IntSlider(
    value=random.randint(0, len(dfs["train"]) - 1) if len(dfs["train"]) else 0,
    min=0,
    max=max(0, len(dfs["train"]) - 1),
    step=1,
    description="Index:",
    continuous_update=False,
    layout=full_row,
)

show_mask_checkbox = widgets.Checkbox(
    value=False,
    description="Show Mask Only",
    layout=checkbox_layout,
)

regenerate_pmap_checkbox = widgets.Checkbox(
    value=False,
    description="Regenerate PMAP",
    layout=checkbox_layout,
)

# Preset selector (radio)
preset_selector = widgets.RadioButtons(
    options=PRESET_NAMES,
    value=DEFAULT_PRESET,
    description="Postproc:",
    layout=widgets.Layout(margin="0 0 0 8px"),
)


# ----------------------------------------
# 3.5) When preset changes, load/lock controls
# ----------------------------------------
smoothing_params = get_postprocessing_params(
    DEFAULT_PRESET if DEFAULT_PRESET != "Custom" else "Custom"
)


def on_preset_change(change=None):
    global smoothing_params

    preset = preset_selector.value
    is_custom = preset == "Custom"
    set_custom_controls_enabled(is_custom)

    path = PRESET_PATHS.get(preset)
    params = load_params_from_file(path, fallback=smoothing_params)

    with suppress_redraw():
        threshold_slider.value = float(params["threshold"])
        closing_radius_slider.value = int(params["closing_radius"])
        min_size_slider.value = int(params["min_size"])
        hole_size_slider.value = float(params["hole_size"])

    update_plot(None)


preset_selector.observe(on_preset_change, names="value")

# Sliders (Custom-only controls)
threshold_slider = widgets.FloatSlider(
    value=smoothing_params["threshold"],
    min=0.0,
    max=1.0,
    step=0.01,
    description="Threshold",
    continuous_update=False,
    layout=full_row,
)
closing_radius_slider = widgets.IntSlider(
    value=smoothing_params["closing_radius"],
    min=0,
    max=20,
    step=1,
    description="Closing R",
    continuous_update=False,
    layout=full_row,
)
min_size_slider = widgets.IntSlider(
    value=smoothing_params["min_size"],
    min=0,
    max=2000,
    step=10,
    description="Min size",
    continuous_update=False,
    layout=full_row,
)
hole_size_slider = widgets.FloatSlider(
    value=smoothing_params["hole_size"],
    min=0.0,
    max=5000,
    step=50,
    description="Hole area",
    continuous_update=False,
    layout=full_row,
)

out = widgets.Output()

CUSTOM_CONTROLS = [
    threshold_slider,
    closing_radius_slider,
    min_size_slider,
    hole_size_slider,
]


def set_custom_controls_enabled(enabled: bool):
    for w in CUSTOM_CONTROLS:
        w.disabled = not enabled


# -------------------------------
# 3) Update slider range when DF changes
# -------------------------------
def update_slider_range(change=None):
    df = dfs[df_selector.value]
    idx_slider.max = max(0, len(df) - 1)
    if idx_slider.value > idx_slider.max:
        idx_slider.value = idx_slider.max


df_selector.observe(update_slider_range, names="value")


# -------------------------------
# 4) Main update function
# -------------------------------
def current_slider_params():
    return {
        "threshold": float(threshold_slider.value),
        "closing_radius": int(closing_radius_slider.value),
        "min_size": int(min_size_slider.value),
        "hole_size": float(hole_size_slider.value),
    }


def update_plot(change=None):
    global smoothing_params

    if _SUPPRESS_REDRAW > 0:
        return

    with out:
        clear_output(wait=True)

        df = dfs[df_selector.value]
        if len(df) == 0:
            print("Selected dataframe is empty.")
            return

        row = df.iloc[idx_slider.value]

        params = current_slider_params()
        smoothing_params = dict(params)

        pmap = None
        if regenerate_pmap_checkbox.value:
            _, pmap = save_pmap(model, row)
            regenerate_pmap_checkbox.value = False

        postprocessing = preset_selector.value
        if postprocessing == "Custom":
            write_params_to_custom(PP_DIR / "Custom.json", smoothing_params)

        plot_sdo(
            row,
            postprocessing=postprocessing,
            show_fits=not show_mask_checkbox.value,
            pmap=pmap,
        )


# -------------------------------
# 5) Hook up callbacks
# -------------------------------
idx_slider.observe(update_plot, names="value")
show_mask_checkbox.observe(update_plot, names="value")
regenerate_pmap_checkbox.observe(update_plot, names="value")
df_selector.observe(update_plot, names="value")

# Sliders should trigger updates only when Custom is active; simplest is to always call
# update_plot, but it will only persist to disk for Custom.
for w in CUSTOM_CONTROLS:
    w.observe(update_plot, names="value")

# -------------------------------
# 6) Display the UI
# -------------------------------
controls = [
    widgets.HBox([idx_slider], layout=full_row),
    widgets.HBox([df_selector, show_mask_checkbox, regenerate_pmap_checkbox, preset_selector], layout=row_wrap),
]
slider_rows = [widgets.HBox([w], layout=full_row) for w in CUSTOM_CONTROLS]

ui = widgets.VBox(controls + slider_rows + [out], layout=full_row)

In [12]:
display(ui)

# Initial draw
update_slider_range(None)
on_preset_change(None)  # sets enabled/disabled + loads Custom if present + draws

# Coronal Hole Area

In [ ]:
# 400 - no oval hole
# 200 — large central hole

In [ ]:
row = df.iloc[4436]
row

In [ ]:
v_min = 300
a = 170
alpha = 0.4


def v(s):
    return v_min + a * (s * 100) ** alpha

In [ ]:
v(ch_rel_area(row, reference_mode=True))

In [ ]:
v(ch_rel_area(row, model, reference_mode=False))

In [ ]:
ch_rel_area(row, reference_mode=True)

In [ ]:
df

In [ ]:
ddf.index = df["20181201_0000":"20181231_2359"].copy().iloc[::3].index

In [ ]:
ddf["ch_unet"] = ddf.apply(lambda x: ch_rel_area(x, reference_mode=False), axis=1)

In [ ]:
ddf["ch_idl"] = ddf.apply(lambda x: ch_rel_area(x, reference_mode=True), axis=1)

In [ ]:
ddf["v_idl"] = ddf["ch_idl"].apply(lambda s: v(s))
ddf["v_unet"] = ddf["ch_unet"].apply(lambda s: v(s))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator


# If you want the index as real datetimes instead of strings, uncomment:
# ddf.index = pd.to_datetime(ddf.index, format="%Y_%m%d")

fig, (ax1, ax2) = plt.subplots(
    nrows=2,
    ncols=1,
    sharex=True,
    figsize=(10, 6),
)

# --- Top panel: ch_idl and unet_idl ---
ax1.plot(ddf.index, ddf["ch_idl"], label="CH Area (Ground Truth)")
ax1.plot(ddf.index, ddf["ch_unet"], label="CH Area (U-Net)")
ax1.set_ylabel("IDL value")
ax1.legend()

# --- Bottom panel: v_idl and v_unet ---
ax2.plot(ddf.index, ddf["v_idl"], label="Velocity (Ground Truth)")
ax2.plot(ddf.index, ddf["v_unet"], label="Velocity (U-Net)")
ax2.set_ylabel("Velocity")
ax2.set_xlabel("Time")
ax2.legend()

ax2.xaxis.set_major_locator(MaxNLocator(nbins=20))  # ~6 major ticks total
plt.setp(ax2.get_xticklabels(), rotation=45, ha="right")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Polarity

In [ ]:
def plot_with_polarity(row, smoothing_params, B_thresh=0.15):
    """
    row: DataFrame row with row.fits_path and row.mask_path and row.hmi_path (JPG)
    smoothing_params: mask-via-model parameters
    """

    aia = prepare_fits(row.fits_path)  # (1024×1024, normalized)
    hmi = prepare_hmi_jpg()  # upscale JPG → match AIA grid

    nn_mask_raw = pmap_to_mask(row, smoothing_params)
    nn_mask = nn_mask_raw > 0.5

    idl_mask_raw = prepare_mask(row.mask_path)
    idl_mask = idl_mask_raw > 0.5

    # polarity masks using pseudo-HMI field
    nn_pos = nn_mask & (hmi >= B_thresh)
    nn_neg = nn_mask & (hmi <= -B_thresh)

    idl_pos = idl_mask & (hmi >= B_thresh)
    idl_neg = idl_mask & (hmi <= -B_thresh)

    # build RGBA overlays
    def make_overlay(pos, neg, alpha=0.5):
        h, w = pos.shape
        rgba_pos = np.zeros((h, w, 4), dtype=np.float32)
        rgba_neg = np.zeros((h, w, 4), dtype=np.float32)

        rgba_pos[..., 0] = 1.0  # red
        rgba_pos[..., 3] = alpha * pos.astype(float)

        rgba_neg[..., 2] = 1.0  # blue
        rgba_neg[..., 3] = alpha * neg.astype(float)

        return rgba_pos, rgba_neg

    nn_overlay_pos, nn_overlay_neg = make_overlay(nn_pos, nn_neg)
    idl_overlay_pos, idl_overlay_neg = make_overlay(idl_pos, idl_neg)

    # plots
    print_distance(row, smoothing_params)
    plt.figure(figsize=(10, 5))

    # ------------------ U-Net ------------------
    plt.subplot(1, 2, 1)
    plt.imshow(aia, cmap=cmap)
    plt.contour(nn_mask.astype(float), levels=[0.5], colors="red")
    plt.imshow(nn_overlay_pos)
    plt.imshow(nn_overlay_neg)
    plt.title("helio-n: red=+, blue=-")
    plt.axis("off")

    # ------------------ IDL ------------------
    plt.subplot(1, 2, 2)
    plt.imshow(aia, cmap=cmap)
    plt.contour(idl_mask.astype(float), levels=[0.5], colors="red")
    plt.imshow(idl_overlay_pos)
    plt.imshow(idl_overlay_neg)
    plt.title("IDL: red=+, blue=-")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
# plot_with_polarity(inf_df.iloc[13385], smoothing_params)

In [ ]:
row = df.dropna().iloc[333]
row

In [ ]:
hmi = sunpy.map.Map(row.hmi_path)
hmi